In [1]:
import pandas as pd
from coypu.utils.query import Query
from coypu.utils.credentials import *
from query.queries import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
import numpy as np
from coypu.utils.functions import json_to_csv
from json import loads, load
import os
from functools import reduce

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

coypu.utils


In [2]:
cmemc_query_imp = Query(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
    
fuseki_infai = Query(fuseki_endpoint_infai, fuseki_user_infai, fuseki_pw_infai, 'basic')

fdquery = Query("http://localhost:5002/sparql", is_fdq=True)

In [ ]:
print(query_3_fdq_desc)
print(query_3_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_3_fdq_ex))

# fdquery.post_query(query_3_fdq_ex)
# fdquery.to_save('../../query_results/', filename='query_3_fdq_ex')

In [ ]:
def load_json(filename):
    with open(filename) as js:
        json_data = load(js)
        return json_data
    
def merge_jsons_to_csv(filepath, save_path, outfile, columns):
    df = reduce(lambda df1, df2: pd.concat([df1, df2]), \
                [json_to_csv(load_json(filepath+filename)['results']['bindings'], save_path, filename, columns, True) for filename in os.listdir(filepath)])
    
    df.to_csv(save_path+outfile, encoding='utf-8', index=False)
    
    
columns=[var+'.value' for var in load_json('../../query_results/query_3/query3_uk.isocode.json')['head']['vars']]
print(columns)
merge_jsons_to_csv('../../query_results/query_3/', '../../query_results/', 'query_3_fdq_ex.csv', columns)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_3_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_3_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

In [3]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port))

df = pd.read_sql(query_3_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_3_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [4]:
df = pd.read_csv('../../query_results/query_3_fdq_ex_sql.csv', encoding='utf-8')

df.head(10)

,isocode,country,year1,gdp_per_capita($)
0,ESP,Spain,1960,396.391397
1,FRA,France,1960,1333.950844
2,GBR,United Kingdom,1960,1397.595420
3,ESP,Spain,1961,450.053271
4,FRA,France,1961,1426.436114
5,GBR,United Kingdom,1961,1472.386364
6,ESP,Spain,1962,520.204674
7,FRA,France,1962,1575.479256
8,GBR,United Kingdom,1962,1525.776526
9,ESP,Spain,1963,609.486938


In [5]:
import plotly.express as px
import plotly.express as px
fig = px.line(df, x='year1', y='gdp_per_capita($)', color='country', symbol="isocode")
fig.update_layout(title_text='hello world')
fig.write_html('../../query_results/query_3_fdq_ex_sql.html')
fig.show()